## Single Decision Tree - Score: 0.83476

Validation set used in final model training.

In [1]:
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import roc_auc_score

pd.options.display.max_rows = 20
pd.options.display.max_columns = 100

### Process Data

In [2]:
# define path to data
path = URLs.path('march-comp')

In [3]:
# define dependent var
dep_var = 'target'

In [4]:
# load tabularpandas object
to = load_pickle(path/'march_tabular.pkl')

In [5]:
# get preds and targets for train and valid set
xs,y = to.train.xs,to.train.y
valid_xs,valid_y = to.valid.xs,to.valid.y

### Create Decision Tree

In [14]:
# concat train and valid sets
full_xs = pd.concat([xs, valid_xs])
full_y = pd.concat([y, valid_y])

In [17]:
# fit DT on train and valid sets
m = DecisionTreeRegressor(min_samples_leaf=25)
m.fit(full_xs, full_y)

DecisionTreeRegressor(min_samples_leaf=25)

### Submit Predictions

In [18]:
# read in test set
test = pd.read_csv(path/'test.csv', low_memory=False)

In [19]:
# convert test set to fastai tabular dataloader
dls = to.dataloaders(bs=1024)
test_dl = dls.test_dl(test, bs=1024)

In [20]:
# checking length of test set & dataloader - make sure same length
len(test), len(test_dl.train.xs)

(200000, 200000)

In [21]:
# get preds
sub_y = m.predict(test_dl.train.xs)

In [22]:
# create submission df
submission = pd.DataFrame({"id": test.id,"target": sub_y})

In [24]:
# save submission to csv
submission.to_csv('DT002.csv', index=False)